### Student Information
Name: 黃俊瑋 

Student ID: 108062308

GitHub ID: [tangerine1202](https://github.com/tangerine1202/DM2022-Lab2-Homework)

Kaggle name: [Chun Wei Huang](https://www.kaggle.com/tangerine1202)

Kaggle private scoreboard snapshot:

![rank 3rd in private leaderboard](img/pic0.png)
*rank 3rd in private leaderboard*

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2022-Lab2-master Repo](https://github.com/keziatamus/DM2022-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm2022-isa5810-lab2-homework) regarding Emotion Recognition on Twitter by [this link](https://www.kaggle.com/t/2b0d14a829f340bc88d2660dc602d4bd). The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   

    Submit your last submission __BEFORE the deadline (Nov. 22th 11:59 pm, Tuesday)_. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the img folder of this repository and rerun the cell Student Information.

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 25th 11:59 pm, Friday)__.

## First Part: Take Home Exercises

- repo link: https://github.com/tangerine1202/DM2022-Lab2-Master

# Second Part: Kaggle Competition

#### Notice: two separated report files

- For clarity, I separated codes into two part, "EDA & data preprocessing" and "training & inference". 

- This notebook is about the "training & inference" part. It directly loaded the preprocessed data from the "EDA & data preprocessing" part.

- For more details about EDA and data preprocessing, please refer to another notebook file at ["./EDA.ipynb"](./EDA.ipynb).

### Summary of the training process

Since training a NLU model requires a lot of resource, it's better to used a pre-trained model. I used the pre-trained model [cardiffnlp/twitter-roberta-base-sentiment-latest](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest) from [HuggingFace](https://huggingface.co/transformers/pretrained_models.html) to fine-tune the model, because this model is pre-trained on Twitter sentiment analysis dataset which is similar to our task. Then, I implemented a custom model to be able to concatenate the external features with the pre-trained model at the last dense layer, see [Custom Model](#Custom-Model).

For my best submission, however, I only used the slightly modified tweet text (see below for details) as only input. I believe that the external features can be helpful to the performance and convergence speed. But it took 40+ hours for the model with only text data to converge. The training time/cost is to huge and the performance of this model already good enough for me, so I decided not to try again with the external features.


*slightly modified tweet text*
- removed "\<LH\>" in the text
- replaced the user mention with "@user" if the times of the user mention is less than 5

*change custom model to concatenate external features*
- uncomment `INPUT_COLUMNS` in [Hyper-parameters](#Hyper-parameters)
- uncomment few lines in `CustomModel` in [Custom Model](#Custom-Model)

(The hyper-parameters used by the best submission are the same as this notebook shown.)

(The output of this notebook is demo with 100 data samples)

In [1]:
from datetime import datetime 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

from tqdm.auto import tqdm

In [2]:
from torch.utils.data import DataLoader
from torch.optim import AdamW

from accelerate import Accelerator
from datasets import Dataset, load_dataset
import evaluate

from transformers import AutoTokenizer
from transformers import AutoModel, AutoConfig, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import pipeline
from transformers import get_scheduler
from transformers import DataCollatorWithPadding
from transformers.modeling_outputs import TokenClassifierOutput
from transformers import XLNetTokenizer, XLNetForSequenceClassification

2022-11-26 18:55:04.909365: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-26 18:55:05.070658: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-26 18:55:05.593939: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/alan/.cuda/lib64:/home/alan/.cuda/extras/CUPTI/lib64:/home/alan/.cuda/lib64:/home/alan/.cuda/extras/CUPTI/lib64:
2022-11-26 18:55:05.594081: W tensorflow/stream_executor/platform/default/dso_loader.

## Hyper-parameters

In [3]:
# data
TRAIN_DATA_PATH = 'data/train_data.pkl'
TRAIN_LABEL_PATH = 'data/train_labels.pkl'
TEST_DATA_PATH = 'data/test_data.pkl'
CHECKPOINTS_PATH = 'checkpoints/'
TEXT_COL_NAME = 'replace_user_text'
EMOTION_NAMES = ['joy', 'anticipation', 'trust' , 'sadness' , 'disgust' , 'fear' , 'surprise', 'anger']
INPUT_COLUMNS = [TEXT_COL_NAME, 'label'] #, *[f'{emo}_ratio' for emo in EMOTION_NAMES]]

# constants
MODEL_NAME = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
NUM_LABELS = 8
CHECKPOINTS_SIZE = 500

# model & training 
EPOCHS = 3
BATCH_SIZE = 16
LR = 1e-5
LR_WARMUP_RATIO = 0.01
TRAIN_SIZE = 145_0000 # total 145_5563
EVAL_SIZE = 2225
TEST_SIZE = 3338

In [4]:
emotion2id = {emotion: i for i, emotion in enumerate(EMOTION_NAMES)}
id2emotion = {i: emotion for i, emotion in enumerate(EMOTION_NAMES)}

## Prepare data

In [5]:
df_train_X = pd.read_pickle(TRAIN_DATA_PATH)
df_train_y = pd.read_pickle(TRAIN_LABEL_PATH)
assert len(df_train_X) == len(df_train_y)
df_train = pd.concat([df_train_X, df_train_y], axis=1)

# numerical labels
df_train['label'] = df_train['emotion'].map(lambda x: emotion2id[x])

df_train.sample(5)

,_score,_crawldate,hashtags,text,lower_tags,joy_ratio,anticipation_ratio,trust_ratio,sadness_ratio,disgust_ratio,fear_ratio,surprise_ratio,anger_ratio,user_mentions,replace_user_text,emotion,label
tweet_id,,,,,,,,,,,,,,,,,
0x34b02b,682,2015-09-15 06:21:50,"[PayAttention, StayWoke]",@funder @jeffgoldesq Mere <LH> ...sh*t's about...,"[#payattention, #staywoke]",0.331984,0.186235,0.076923,0.178138,0.072874,0.036437,0.085020,0.032389,"[funder, jeffgoldesq]",@funder @user Mere ...sh*t's about to hit the...,surprise,6
0x2ab860,836,2016-02-24 11:09:26,"[WordPlay, beautiful]",#WordPlay helps turn something hateful into so...,"[#wordplay, #beautiful]",0.962721,0.005232,0.015043,0.002616,0.009810,0.001962,0.001308,0.001308,[],#WordPlay helps turn something hateful into so...,joy,0
0x23a81a,145,2016-01-20 08:02:20,[YouTuber],a big shout out to anyone who is either like c...,[#youtuber],0.537415,0.170068,0.115646,0.061224,0.044218,0.013605,0.040816,0.017007,[],a big shout out to anyone who is either like c...,joy,0
0x24cbd2,790,2017-02-04 12:09:20,"[RickPitino, FBI, NCAABribery]",Anyone that believes #RickPitino knew nothing ...,"[#rickpitino, #fbi, #ncaabribery]",0.070175,0.122807,0.149123,0.307018,0.166667,0.052632,0.078947,0.052632,[],Anyone that believes #RickPitino knew nothing ...,anticipation,1
0x3142a3,182,2017-01-06 23:24:39,[blessings],Typos can't count on that 🙏🏽 <LH> #blessings,[#blessings],0.456048,0.264463,0.262960,0.005259,0.001503,0.003757,0.004508,0.001503,[],Typos can't count on that 🙏🏽 #blessings,anticipation,1


### Tokenize the text column

In [6]:
def tokenize_function(examples):
    return tokenizer(examples[TEXT_COL_NAME], padding="max_length", truncation=True, max_length=512)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
dataset = Dataset.from_pandas(df_train[INPUT_COLUMNS])
dataset

Dataset({
    features: ['replace_user_text', 'label', 'tweet_id'],
    num_rows: 1455563
})

In [8]:
# tokenize the text column
tokenized_datasets = dataset.map(tokenize_function, batched=True)
# saved the tokenized datasets, so we don't need to tokenize again if we do not change the text column
tokenized_datasets.save_to_disk(f'data/{MODEL_NAME}_tokenized_datasets')

A Jupyter Widget

In [9]:
# load the tokenized datasets
tokenized_datasets = Dataset.load_from_disk(f'data/{MODEL_NAME}_tokenized_datasets')

### Transform to the hugging face format

In [10]:
# process dataset features to fit the requirement of the hugging face model
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets = tokenized_datasets.remove_columns([TEXT_COL_NAME, 'tweet_id'])
tokenized_datasets.set_format("torch")
tokenized_datasets

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1455563
})

### Split the data

In [11]:
# split the dataset
small_train_dataset = tokenized_datasets.shuffle(seed=42).select(range(TRAIN_SIZE))
small_eval_dataset = tokenized_datasets.shuffle(seed=42).select(range(TRAIN_SIZE, TRAIN_SIZE + EVAL_SIZE))
test_dataset = tokenized_datasets.shuffle(seed=42).select(range(TRAIN_SIZE + EVAL_SIZE, TRAIN_SIZE + EVAL_SIZE + TEST_SIZE))
# NOTE: do not shuffle the training set in dataloader again, in order to keep the same shuffle random seed
train_dataloader = DataLoader(small_train_dataset, batch_size=BATCH_SIZE, collate_fn=data_collator)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=BATCH_SIZE, collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=data_collator)

print(f'training size:\t{len(small_train_dataset)}\t{len(small_train_dataset) / 1455563 * 100 :.3f}%')
print(f'eval     size:\t{len(small_eval_dataset)}\t{len(small_eval_dataset) / 1455563 * 100 :.3f}%')
print(f'test     size:\t{len(test_dataset)}\t{len(test_dataset) / 1455563 * 100 :.3f}%')

Loading cached shuffled indices for dataset at data/cardiffnlp/twitter-roberta-base-sentiment-latest_tokenized_datasets/cache-264420d4d8242261.arrow
Loading cached shuffled indices for dataset at data/cardiffnlp/twitter-roberta-base-sentiment-latest_tokenized_datasets/cache-264420d4d8242261.arrow
Loading cached shuffled indices for dataset at data/cardiffnlp/twitter-roberta-base-sentiment-latest_tokenized_datasets/cache-264420d4d8242261.arrow


training size:	100	0.007%
eval     size:	100	0.007%
test     size:	100	0.007%


## Prepare model

### Custom Model

- combining external feature to the BERT model

In [12]:
# Build custom model to *combine the external features* into the hugging face model
# ref: https://towardsdatascience.com/adding-custom-layers-on-top-of-a-hugging-face-model-f1ccdfc257bd
class CustomModel(torch.nn.Module):
  def __init__(self, checkpoint, num_labels): 
    super(CustomModel,self).__init__() 
    self.num_labels = num_labels 

    # Load Model with given checkpoint and extract its body
    self.model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
    self.seq_dropout = torch.nn.Dropout(0.3) 
    # self.features_dropout = torch.nn.Dropout(0.0)
    # self.classifier = torch.nn.Linear(768 + NUM_LABELS, num_labels) # load and initialize weights
    self.classifier = torch.nn.Linear(768, num_labels) # load and initialize weights

  def forward(self, 
    input_ids=None, attention_mask=None, labels=None,
    # joy_ratio=None, anticipation_ratio=None, trust_ratio=None, sadness_ratio=None, disgust_ratio=None, fear_ratio=None, surprise_ratio=None, anger_ratio=None
  ):
    # Extract outputs from the body
    # pretrained_outputs[0]=last hidden state
    pretrained_outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
    seq_outputs = self.seq_dropout(pretrained_outputs[0])[:, 0, :].view(-1, 768) # seq shape torch.Size([16, 768])

    # Add emotion ratios
    # emo_ratio_outputs = torch.stack([joy_ratio, anticipation_ratio, trust_ratio, sadness_ratio, disgust_ratio, fear_ratio, surprise_ratio, anger_ratio], dim=1)
    # features_outputs = self.features_dropout(emo_ratio_outputs) # emo_ratio shape torch.Size([16, 8])

    # Concatenate
    # outputs = torch.cat((seq_outputs, features_outputs), dim=1)
    outputs = seq_outputs
    logits = self.classifier(outputs) # calculate losses
    
    loss = None
    if labels is not None:
      loss_fct = torch.nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
    
    return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=pretrained_outputs.hidden_states ,attentions=pretrained_outputs.attentions)
  
  def save_checkpoint(self, steps):
    torch.save(self.state_dict(), f'{CHECKPOINTS_PATH}{steps}.pt')
  

In [13]:
# instantiate the model
if os.path.isfile(f'model/{MODEL_NAME}-custom-{TRAIN_SIZE}') and input('Use pre-trained model? (y/n)') == 'y':
  model = torch.load(f'model/{MODEL_NAME}-custom-{TRAIN_SIZE}')
  print('Using pre-trained model')
else:
  model = CustomModel(MODEL_NAME, NUM_LABELS)
  print('Instantiate new model')
model.to(device)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaModel: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Instantiate new model


CustomModel(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,),

In [14]:
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-2)

# linear learning rate scheduler with warmup
num_training_steps = EPOCHS * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=int(num_training_steps*LR_WARMUP_RATIO), num_training_steps=num_training_steps
)

In [15]:
f1_metric = evaluate.load("f1")
acu_metric = evaluate.load("accuracy")

def compute_metrics():
    f1_macro = f1_metric.compute(average='macro')['f1']
    acu = acu_metric.compute()['accuracy']
    return {'f1_macro': f1_macro, 'acu': acu}


In [16]:
# hugging face accelerator
accelerator = Accelerator()

device = accelerator.device
model.to(device)

model, optimizer, train_dataloader, lr_scheduler, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, lr_scheduler, eval_dataloader
)

## Training

In [17]:
progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(int(num_training_steps / CHECKPOINTS_SIZE) * len(eval_dataloader)))

best_f1 = 0
early_stop_cnt = 0
steps = 0
cumulative_loss = 0
for epoch in range(EPOCHS):
  model.train()
  for batch in train_dataloader:
      optimizer.zero_grad()
      # batch = {k: v.to(device) for k, v in batch.items()}
      batch = {k: v for k, v in batch.items()}  # accelerator
      outputs = model(**batch)
      loss = outputs.loss
      # loss.backward()
      accelerator.backward(loss)  # accelerator
      
      optimizer.step()
      lr_scheduler.step()
      progress_bar_train.update(1)

      cumulative_loss += loss.item()
      steps += 1

      if steps % CHECKPOINTS_SIZE == 0:
        print(f'step {steps} training loss: {cumulative_loss / CHECKPOINTS_SIZE}')
        cumulative_loss = 0
        model.save_checkpoint(steps)

        model.eval()
        for batch in eval_dataloader:
          # batch = {k: v.to(device) for k, v in batch.items()}
          batch = {k: v for k, v in batch.items()}  # accelerator
          with torch.no_grad():
              outputs = model(**batch)

          logits = outputs.logits
          predictions = torch.argmax(logits, dim=-1)
          f1_metric.add_batch(predictions=predictions, references=batch["labels"])
          acu_metric.add_batch(predictions=predictions, references=batch["labels"])
          progress_bar_eval.update(1)
          
        metrics = compute_metrics()
        print(metrics)

        # early stop when f1 score drop significantly
        if metrics['f1_macro'] < (best_f1 - 0.1):
          early_stop_cnt += 1
          print(f'early stop cnt = {early_stop_cnt}')
          if early_stop_cnt >= 3:
            print('Early stop')
            break

        best_f1 = max(best_f1, metrics['f1_macro'])
        model.train()

A Jupyter Widget

A Jupyter Widget

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [18]:
# save model
saved_path = f'model/{datetime.now().strftime("%Y%m%d-%H%M%S")}'
torch.save(model, saved_path)
print(f'Saved model to {saved_path}')

Saved model to model/20221126-185821


## Evaluation

In [19]:
f1_metric = evaluate.load("f1")
acu_metric = evaluate.load("accuracy")

progress_bar_test = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    f1_metric.add_batch(predictions=predictions, references=batch["labels"])
    acu_metric.add_batch(predictions=predictions, references=batch["labels"])
    progress_bar_test.update(1)

print(compute_metrics())

A Jupyter Widget

{'f1_macro': 0.13203794369645042, 'acu': 0.47}


## Inference

In [20]:
# model = CustomModel(MODEL_NAME, NUM_LABELS).to(device)
# model.load_state_dict(torch.load(f'{CHECKPOINTS_PATH}{195000}.pt'))
# model = torch.load(f'model/20221124-235147')

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaModel: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
TEST_BATCH_SIZE = 40

TEST_INPUT_COLUMNS = INPUT_COLUMNS.copy()
if 'label' in TEST_INPUT_COLUMNS:
  TEST_INPUT_COLUMNS.remove('label')

In [22]:
df_test = pd.read_pickle(TEST_DATA_PATH)
df_test = df_test[TEST_INPUT_COLUMNS]
df_test.sample(5)

,replace_user_text
tweet_id,
0x1fadea,So @Qantas says it's @user @user fault that no...
0x1e64e4,The great thing about faith in God is that it ...
0x353889,"@eucopresident Donald Tusk, @EU_Commission Jea..."
0x2d4320,Maybe the Wall was the friends we made along t...
0x231e50,@colin_w_ I specifically answered to get Ursul...


In [23]:
test_dataset = Dataset.from_pandas(df_test[TEST_INPUT_COLUMNS])
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.remove_columns([TEXT_COL_NAME, 'tweet_id'])
test_dataset.set_format(type='torch')
test_dataset

A Jupyter Widget

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 411972
})

In [24]:
test_dataloader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, collate_fn=data_collator)

In [25]:
df_result = pd.DataFrame({'id': df_test.index})
df_result['label'] = np.zeros(len(df_result)) - 1
gpu_tensor = torch.tensor(df_result['label'].values, dtype=torch.long, device=device)

progress_bar_test = tqdm(range(len(test_dataloader)))

idx = 0
model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    gpu_tensor[idx:idx+len(predictions)] = predictions
    idx += len(predictions)
    progress_bar_test.update(1)

df_result['label'] = gpu_tensor.cpu().numpy()

# convert label to emotion
df_result['emotion'] = df_result['label'].map(lambda x: id2emotion[x])

df_result.head(3)

A Jupyter Widget

KeyboardInterrupt: 

In [ ]:
saved_path = f'submission/{MODEL_NAME}-custom-{datetime.now().strftime("%Y%m%d-%H%M%S")}.csv'
df_result[['id', 'emotion']].to_csv(saved_path, index=False)
print(saved_path)

submission/cardiffnlp/twitter-roberta-base-sentiment-latest-custom-20221119-234512.csv


50000